# Firewall Log Analysis

## Step 1
- Importing json and pandas libraries for data manipulation and analysis.
- Having a look at the entire dataset to understand its structure and contents.
- Then I use df.info() to get a concise summary of the DataFrame.

In [79]:
import json
import pandas as pd

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Try line-delimited JSON first, then fall back to a single JSON object/array.
try:
    df = pd.read_json("lab_2_firewall_log.json", lines=True)
except ValueError:
    with open("lab_2_firewall_log.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.DataFrame(data)

df

,@timestamp,cisco.asa.destination_interface,cisco.asa.message_id,cisco.asa.rule_name,cisco.asa.source_interface,destination.address,destination.ip,destination.port,event.action,event.category,event.code,event.dataset,event.kind,event.module,event.original,event.outcome,event.severity,event.timezone,event.type,fileset.name,input.type,log.file.path,log.level,log.offset,network.community_id,network.iana_number,network.transport,observer.egress.interface.name,observer.ingress.interface.name,observer.product,observer.type,observer.vendor,related.ip,service.type,source.address,source.ip,source.port,tags,cisco.asa.suffix,host.hostname,observer.hostname,related.hosts,cisco.asa.connection_id,cisco.asa.mapped_destination_ip,cisco.asa.mapped_destination_port,cisco.asa.mapped_source_ip,cisco.asa.mapped_source_port,destination.nat.port,network.direction,source.nat.ip,destination.nat.ip,event.duration,event.end,event.start,network.bytes,cisco.asa.destination_username,cisco.asa.source_username,destination.user.name,related.user,source.user.name,user.name,network.protocol,process.name,cisco.asa.mapped_destination_host,destination.domain,event.reason,cisco.asa.icmp_code,cisco.asa.icmp_type,server.domain,source.nat.port,cisco.asa.threat_category,cisco.asa.threat_level,url.original,url.path,url.domain,url.scheme,url.extension,cisco.asa.termination_user,destination.geo.city_name,destination.geo.continent_name,destination.geo.country_iso_code,destination.geo.country_name,destination.geo.location.lat,destination.geo.location.lon,destination.geo.region_iso_code,destination.geo.region_name,destination.as.number,destination.as.organization.name,source.as.number,source.as.organization.name,source.user.domain,source.geo.city_name,source.geo.continent_name,source.geo.country_iso_code,source.geo.country_name,source.geo.location.lat,source.geo.location.lon,source.geo.region_iso_code,source.geo.region_name,destination.user.domain,cisco.asa.termination_initiator,cisco.asa.tunnel_type,network.type,source.domain,log.syslog.facility.code,log.syslog.priority,log.syslog.severity.code,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
0,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],106023.0,cisco.asa,event,cisco,"%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group ""acl_dmz"" [0xe3aab522, 0x0]",success,4.0,-02:00,"[connection, denied]",asa,log,sample.log,warning,0.0,1:3NxcSu9jwJUYIYwJ2TO4TSNnPX8=,6.0,tcp,outside,dmz,asa,firewall,Cisco,"[10.1.2.30, 192.0.0.8]",cisco,10.1.2.30,10.1.2.30,63016.0,"[cisco-asa, forwarded]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],106023.0,cisco.asa,event,cisco,"%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group ""acl_dmz"" [0xe3aab522, 0x0]",success,4.0,-02:00,"[connection, denied]",asa,log,sample.log,warning,139.0,1:3NxcSu9jwJUYIYwJ2TO4TSNnPX8=,6.0,tcp,outside,dmz,asa,firewall,Cisco,"[10.1.2.30, 192.0.0.8]",cisco,10.1.2.30,10.1.2.30,63016.0,"[cisco-asa, forwarded]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-04-15T13:34:34.000Z,outside,106100,acl_in,inside,192.0.0.89,,2000.0,firewall-rule,[network],106100.0,cisco.asa,event,cisco,"%ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Columns: 115 entries, @timestamp to bytes
dtypes: float64(24), object(91)
memory usage: 125.0+ KB


In [81]:
# Count how many times each column appears with a non-null value
column_counts = df.notna().sum().sort_values(ascending=False)
column_counts

@timestamp                           89
cisco.asa.message_id                 89
event.code                           89
event.severity                       89
event.action                         89
event.module                         89
event.dataset                        89
event.original                       89
tags                                 89
observer.vendor                      89
source.address                       89
observer.type                        89
observer.product                     89
service.type                         89
log.offset                           89
log.level                            89
log.file.path                        89
input.type                           89
fileset.name                         89
source.ip                            88
related.ip                           88
destination.address                  88
event.kind                           87
event.category                       87
event.type                           87


The above tells us the events are primarily in 2 formats
- The first 89 rows follow the first format with fields like @timestamp etc.
- The last 50 rows follow a different format with fields like timestamp

So we might need to treat them differently

## Timestamp Related Data Cleaning
- The dataset has two different timestamp formats, so we need to normalize them into a single format
- I define a function get_normalized_timestamp that checks for the presence of @timestamp and timestamp fields and returns the appropriate one
- Then I apply this function to create a new column normalized_timestamp
- Finally, I convert the normalized_timestamp column to datetime format using pd.to_datetime, handling any errors by coercing them to NaT (Not a Time) and display the relevant columns to verify the results.

In [82]:

import numpy as np

# Normalize timestamp: use '@timestamp' if present, else 'timestamp', else NaN
def get_normalized_timestamp(row):
    if pd.notna(row.get('@timestamp')) and row.get('@timestamp') != '':
        return row.get('@timestamp')
    elif pd.notna(row.get('timestamp')) and row.get('timestamp') != '':
        return row.get('timestamp')
    else:
        return np.nan

# First create a new column 'normalized_timestamp' using the function
df['normalized_timestamp'] = df.apply(get_normalized_timestamp, axis=1)
# Next convert the 'normalized_timestamp' column to datetime format, handling any errors by coercing them to NaT (Not a Time)
# The utc=True argument ensures that the resulting datetime objects are timezone-aware and set to UTC, 
# This is important for consistent time handling across different timezones. And we don't have to do any further timezone conversion later.
df['normalized_timestamp'] = pd.to_datetime(df['normalized_timestamp'], errors='coerce', utc=True)
df[['@timestamp', 'timestamp', 'normalized_timestamp', 'event.timezone']]

,@timestamp,timestamp,normalized_timestamp,event.timezone
0,2013-04-15T09:36:50.000-02:00,NaN,2013-04-15 11:36:50+00:00,-02:00
1,2013-04-15T09:36:50.000-02:00,NaN,2013-04-15 11:36:50+00:00,-02:00
2,2014-04-15T13:34:34.000Z,NaN,2014-04-15 13:34:34+00:00,NaN
3,2013-04-24T16:00:28.000-02:00,NaN,2013-04-24 18:00:28+00:00,-02:00
4,2013-04-24T16:00:27.000-02:00,NaN,2013-04-24 18:00:27+00:00,-02:00
5,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 14:59:50+00:00,-02:00
6,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 14:59:50+00:00,-02:00
7,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 14:59:50+00:00,-02:00
8,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 14:59:50+00:00,-02:00
9,2013-04-29T12:59:50.000-02:00,NaN,2013-04-29 14:59:50+00:00,-02:00


In [83]:
# Make sure all possible timestamp formats are parsed
df['normalized_timestamp'] = pd.to_datetime(
    df['normalized_timestamp'], errors='coerce', utc=True
)

# Only drop rows where timestamp could not be parsed at all
df_valid_time = df[df['normalized_timestamp'].notna()].copy()

# Group by 24-hour periods (date only, in UTC)
df_valid_time['date'] = df_valid_time['normalized_timestamp'].dt.floor('D')

# Now group and analyze as before
grouped_by_day = df_valid_time.groupby('date')
events_per_day = grouped_by_day.size().reset_index(name='event_count')
events_per_day

,date,event_count
0,2009-11-16 00:00:00+00:00,3
1,2011-06-04 00:00:00+00:00,1
2,2012-08-16 00:00:00+00:00,1
3,2013-04-15 00:00:00+00:00,2
4,2013-04-24 00:00:00+00:00,2
5,2013-04-29 00:00:00+00:00,10
6,2013-04-30 00:00:00+00:00,21
7,2014-04-15 00:00:00+00:00,1
8,2014-09-12 00:00:00+00:00,8
9,2015-01-14 00:00:00+00:00,4


In [84]:
bad_rows = df[df['normalized_timestamp'].isna()][['@timestamp', 'timestamp']]
bad_rows

,@timestamp,timestamp
17,,NaN
40,,NaN
56,,NaN
63,,NaN
73,,NaN
78,,NaN


## Data Cleaning: Split Dataset and Select Relevant Columns

The dataset contains two distinct formats:
- **First 89 rows**: ECS format with nested fields (source.address, destination.ip, etc.)
- **Last 50 rows**: Simple format with flat fields (source_ip, dest_ip, etc.)

We'll split the dataset and keep only relevant columns for each format.

In [85]:
# Split dataset into two parts
df1 = df.iloc[:89].copy()  # First 89 rows
df2 = df.iloc[-50:].copy()  # Last 50 rows

print(f"Dataset 1 shape: {df1.shape}")
print(f"Dataset 2 shape: {df2.shape}")

Dataset 1 shape: (89, 116)
Dataset 2 shape: (50, 116)


In [86]:
# Process first 89 rows - ECS format
# Combine source.address and source.ip into one column
df1['source_combined'] = df1.apply(
    lambda row: row.get('source.address') if pd.notna(row.get('source.address')) 
    else row.get('source.ip'), axis=1
)

# Combine destination.address and destination.ip into one column
df1['destination_combined'] = df1.apply(
    lambda row: row.get('destination.address') if pd.notna(row.get('destination.address')) 
    else row.get('destination.ip'), axis=1
)

# Select relevant columns
df1_clean = df1[[
    'normalized_timestamp',
    'source_combined',
    'destination_combined',
    'source.port',
    'destination.port',
    'related.ip',
    'network.transport',
    'event.original',
    'event.outcome',
    'event.severity',
    'event.type',
    'log.level'
]].copy()

# Rename columns for clarity
df1_clean.columns = [
    'timestamp',
    'source',
    'destination',
    'source_port',
    'destination_port',
    'related_ip',
    'protocol',
    'original_event',
    'outcome',
    'severity',
    'event_type',
    'log_level'
]

df1_clean

,timestamp,source,destination,source_port,destination_port,related_ip,protocol,original_event,outcome,severity,event_type,log_level
0,2013-04-15 11:36:50+00:00,10.1.2.30,192.0.0.8,63016.0,53.0,"[10.1.2.30, 192.0.0.8]",tcp,"%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group ""acl_dmz"" [0xe3aab522, 0x0]",success,4.0,"[connection, denied]",warning
1,2013-04-15 11:36:50+00:00,10.1.2.30,192.0.0.8,63016.0,53.0,"[10.1.2.30, 192.0.0.8]",tcp,"%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group ""acl_dmz"" [0xe3aab522, 0x0]",success,4.0,"[connection, denied]",warning
2,2014-04-15 13:34:34+00:00,10.1.2.16,192.0.0.89,2241.0,2000.0,"[10.1.2.16, 192.0.0.89]",tcp,"%ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-cnt 1 first hit [0x71a87d94, 0x0]",success,5.0,"[allowed, connection]",notification
3,2013-04-24 18:00:28+00:00,172.29.2.101,192.0.2.10,1039.0,53.0,"[172.29.2.101, 192.0.2.10]",udp,"%ASA-6-106100: access-list inside denied udp inside/172.29.2.101(1039) -> outside/192.0.2.10(53) hit-cnt 1 first hit [0xd820e56a, 0x0]",success,6.0,"[connection, denied]",informational
4,2013-04-24 18:00:27+00:00,172.29.2.3,192.0.2.57,1065.0,53.0,"[172.29.2.3, 192.0.2.57]",udp,"%ASA-6-106100: access-list inside permitted udp inside/172.29.2.3(1065) -> outside/192.0.2.57(53) hit-cnt 144 300-second interval [0xe982c7a4, 0x0]",success,6.0,"[allowed, connection]",informational
5,2013-04-29 14:59:50+00:00,10.123.3.42,192.0.2.130,4952.0,12834.0,"[10.123.3.42, 192.0.2.130]",tcp,%ASA-6-305011: Built dynamic TCP translation from outside:10.123.3.42/4952 to outside:192.0.2.130/12834,NaN,6.0,[info],informational
6,2013-04-29 14:59:50+00:00,192.0.2.43,10.123.3.42,443.0,4952.0,"[10.123.3.42, 192.0.2.43]",tcp,%ASA-6-302013: Built outbound TCP connection 89743274 for outside:192.0.2.43/443 (192.0.2.43/443) to outside:10.123.3.42/4952 (10.123.3.42/12834),NaN,6.0,[info],informational
7,2013-04-29 14:59:50+00:00,10.123.1.35,192.0.2.130,52925.0,25882.0,"[10.123.1.35, 192.0.2.130]",udp,%ASA-6-305011: Built dynamic UDP translation from outside:10.123.1.35/52925 to outside:192.0.2.130/25882,NaN,6.0,[info],informational
8,2013-04-29 14:59:50+00:00,192.0.2.222,10.123.1.35,53.0,52925.0,"[10.123.1.35, 192.0.2.222, 192.0.2.43]",udp,%ASA-6-302015: Built outbound UDP connection 89743275 for outside:192.0.2.222/53 (192.0.2.43/53) to outside:10.123.1.35/52925 (10.123.1.35/25882),NaN,6.0,[info],informational
9,2013-04-29 14:59:50+00:00,10.123.3.42,192.0.2.130,4953.0,45392.0,"[10.123.3.42, 192.0.2.130]",tcp,%ASA-6-305011: Built dynamic TCP translation from outside:10.123.3.42/4953 to outside:192.0.2.130/45392,NaN,6.0,[info],informational


## Having a look at the Ports, to find suspicious activity related to non-standard ports

In [87]:
from collections import Counter
# Analyze most common ports: total, as source, as destination, and percent of all rows
def flatten_ports(col):
    ports = []
    for v in col:
        if isinstance(v, list):
            ports.extend(v)
        elif pd.notna(v):
            ports.append(v)
    return ports
total_rows = len(df)

src_ports = flatten_ports(df['src_port'])
dest_ports = flatten_ports(df['dest_port'])
all_ports = src_ports + dest_ports
port_counter = Counter(all_ports)
src_port_counter = Counter(src_ports)
dest_port_counter = Counter(dest_ports)
port_stats = []

for port, count in port_counter.most_common():
    src_count = src_port_counter.get(port, 0)
    dest_count = dest_port_counter.get(port, 0)
    percent = (count / total_rows) * 100
    port_stats.append({'port': port, 'total': count, 'source': src_count, 'destination': dest_count, 'percent': percent})
port_stats_df = pd.DataFrame(port_stats)
port_stats_df.sort_values('total', ascending=False).head(20)

,port,total,source,destination,percent
0,443.0,35,0,35,25.179856
1,22.0,10,0,10,7.194245
2,4444.0,5,0,5,3.597122
3,20794.0,1,1,0,0.719424
4,57972.0,1,1,0,0.719424
5,45292.0,1,1,0,0.719424
6,32148.0,1,1,0,0.719424
7,31514.0,1,1,0,0.719424
8,28435.0,1,1,0,0.719424
9,36349.0,1,1,0,0.719424


## Splitting the Dataset in 2 to make it easier to analyze the different formats separately

In [88]:
# Process last 50 rows - Simple format
df2_clean = df2[[
    'normalized_timestamp',
    'src_ip',
    'dest_ip',
    'src_port',
    'dest_port',
    'protocol',
    'action',
    'bytes'
]].copy()

# Rename timestamp column for consistency
df2_clean.columns = [
    'timestamp',
    'src_ip',
    'dest_ip',
    'src_port',
    'dest_port',
    'protocol',
    'action',
    'bytes'
]

df2_clean

,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
89,2026-02-02 17:23:49.974906+00:00,10.0.0.27,8.8.8.8,20794.0,443.0,TCP,ALLOW,1910.0
90,2026-02-02 17:24:49.974906+00:00,10.0.0.27,8.8.8.8,57972.0,443.0,TCP,ALLOW,1612.0
91,2026-02-02 17:25:49.974906+00:00,10.0.0.19,8.8.8.8,45292.0,443.0,TCP,ALLOW,1813.0
92,2026-02-02 17:26:49.974906+00:00,10.0.0.43,8.8.8.8,32148.0,443.0,TCP,ALLOW,803.0
93,2026-02-02 17:27:49.974906+00:00,10.0.0.37,8.8.8.8,31514.0,443.0,TCP,ALLOW,987.0
94,2026-02-02 17:28:49.974906+00:00,10.0.0.40,8.8.8.8,28435.0,443.0,TCP,ALLOW,4321.0
95,2026-02-02 17:29:49.974906+00:00,10.0.0.11,8.8.8.8,36349.0,443.0,TCP,ALLOW,3719.0
96,2026-02-02 17:30:49.974906+00:00,10.0.0.45,8.8.8.8,39399.0,443.0,TCP,ALLOW,4293.0
97,2026-02-02 17:31:49.974906+00:00,10.0.0.13,8.8.8.8,48968.0,443.0,TCP,ALLOW,2891.0
98,2026-02-02 17:32:49.974906+00:00,10.0.0.15,8.8.8.8,52460.0,443.0,TCP,ALLOW,4847.0


In [89]:
# Display info about both cleaned datasets
print("=" * 60)
print("Dataset 1 (First 89 rows)")
print("=" * 60)
print(df1_clean.info())
print("\n")

print("=" * 60)
print("Dataset 2 (Last 50 rows)")
print("=" * 60)
print(df2_clean.info())

Dataset 1 (First 89 rows)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   timestamp         83 non-null     datetime64[ns, UTC]
 1   source            89 non-null     object             
 2   destination       88 non-null     object             
 3   source_port       69 non-null     float64            
 4   destination_port  69 non-null     float64            
 5   related_ip        88 non-null     object             
 6   protocol          71 non-null     object             
 7   original_event    89 non-null     object             
 8   outcome           54 non-null     object             
 9   severity          89 non-null     float64            
 10  event_type        87 non-null     object             
 11  log_level         89 non-null     object             
dtypes: datetime64[ns, UTC](1), float64(3), o

# DataFrame 1 Analysis
- This seems like a Firewall log dataset
- For the first 89 rows, we select the relevant columns and rename them for consistency.
- We also drop the original timestamp columns since we have the normalized timestamp.
- Finally, we display the cleaned dataset to verify the changes.
## Issues noted on first glance with help of AI:
- Port(53) Suspicious DNS traffic, Looking Possible DNS probing or reconnaissance activity
- Port(137) Analyzing NetBIOS traffic from external sources, which is often associated with reconnaissance or exploitation attempts.
- Blaclisted Malware Traffic, where original_event string contains the word "blacklist" or "malware"
- Non-Standard Port 40443 Outbound Traffic, port 40443 is not commonly used and may indicate suspicious activity
- IP spoofing attempts where source IP = 0.0.0.0
- High Severity Events severity > 3

### Analysis 1: Port 53 - DNS Traffic Denials
Analyzing repeated denied actions for inbound DNS traffic (port 53) which may indicate DNS probing or reconnaissance activity.

In [90]:
# Filter for port 53 DNS traffic with denied outcomes
dns_denied = df1_clean[
    ((df1_clean['source_port'] == 53) | (df1_clean['destination_port'] == 53))
].copy()

print(f"Total DNS denied events: {len(dns_denied)}")
print(f"\nBreakdown by source:")
print(dns_denied.groupby('source').size().sort_values(ascending=False))
print(f"\nBreakdown by destination:")
print(dns_denied.groupby('destination').size().sort_values(ascending=False))
print("\n" + "="*60)
print("Sample DNS denied events:")
print("="*60)
# dns_denied[['timestamp', 'source', 'destination', 'source_port', 'destination_port', 'outcome', 'severity']]
dns_denied[['timestamp', 'source', 'destination', 'source_port', 'destination_port', 'related_ip', 'protocol', 'outcome', 'severity', 'event_type', 'original_event', 'log_level']]

Total DNS denied events: 13

Breakdown by source:
source
192.0.2.222      3
10.1.2.30        2
192.168.1.33     2
89.160.20.156    2
192.0.0.66       1
172.29.2.3       1
172.29.2.101     1
192.0.2.66       1
dtype: int64

Breakdown by destination:
destination
10.123.1.35    3
192.0.0.12     2
192.168.0.1    2
192.0.0.8      2
10.1.2.60      1
10.1.5.60      1
192.0.2.10     1
192.0.2.57     1
dtype: int64

Sample DNS denied events:


,timestamp,source,destination,source_port,destination_port,related_ip,protocol,outcome,severity,event_type,original_event,log_level
0,2013-04-15 11:36:50+00:00,10.1.2.30,192.0.0.8,63016.0,53.0,"[10.1.2.30, 192.0.0.8]",tcp,success,4.0,"[connection, denied]","%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group ""acl_dmz"" [0xe3aab522, 0x0]",warning
1,2013-04-15 11:36:50+00:00,10.1.2.30,192.0.0.8,63016.0,53.0,"[10.1.2.30, 192.0.0.8]",tcp,success,4.0,"[connection, denied]","%ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group ""acl_dmz"" [0xe3aab522, 0x0]",warning
3,2013-04-24 18:00:28+00:00,172.29.2.101,192.0.2.10,1039.0,53.0,"[172.29.2.101, 192.0.2.10]",udp,success,6.0,"[connection, denied]","%ASA-6-106100: access-list inside denied udp inside/172.29.2.101(1039) -> outside/192.0.2.10(53) hit-cnt 1 first hit [0xd820e56a, 0x0]",informational
4,2013-04-24 18:00:27+00:00,172.29.2.3,192.0.2.57,1065.0,53.0,"[172.29.2.3, 192.0.2.57]",udp,success,6.0,"[allowed, connection]","%ASA-6-106100: access-list inside permitted udp inside/172.29.2.3(1065) -> outside/192.0.2.57(53) hit-cnt 144 300-second interval [0xe982c7a4, 0x0]",informational
8,2013-04-29 14:59:50+00:00,192.0.2.222,10.123.1.35,53.0,52925.0,"[10.123.1.35, 192.0.2.222, 192.0.2.43]",udp,NaN,6.0,[info],%ASA-6-302015: Built outbound UDP connection 89743275 for outside:192.0.2.222/53 (192.0.2.43/53) to outside:10.123.1.35/52925 (10.123.1.35/25882),informational
11,2013-04-29 14:59:50+00:00,192.0.2.222,10.123.1.35,53.0,52925.0,"[10.123.1.35, 192.0.2.222]",udp,NaN,6.0,"[connection, end]",%ASA-6-302016: Teardown UDP connection 89743275 for outside:192.0.2.222/53 to inside:10.123.1.35/52925 duration 1:23:45 bytes 140,informational
12,2013-04-29 14:59:50+00:00,192.0.2.222,10.123.1.35,53.0,52925.0,"[10.123.1.35, 192.0.2.222]",udp,NaN,6.0,"[connection, end]",%ASA-6-302016: Teardown UDP connection 666 for outside:192.0.2.222/53 user1 to inside:10.123.1.35/52925 user2 duration 10:00:00 bytes 9999999,informational
16,2013-04-30 11:22:33+00:00,192.0.0.66,10.1.2.60,12981.0,53.0,"[10.1.2.60, 192.0.0.66]",udp,success,2.0,"[connection, denied]",%ASA-2-106007: Deny inbound UDP from 192.0.0.66/12981 to 10.1.2.60/53 due to DNS Query,critical
28,2013-04-30 11:23:03+00:00,192.0.2.66,10.1.5.60,12981.0,53.0,"[10.1.5.60, 192.0.2.66]",udp,success,2.0,"[connection, denied]",%ASA-2-106007: Deny inbound UDP from 192.0.2.66/12981 to 10.1.5.60/53 due to DNS Query,critical
40,NaT,192.168.1.33,192.0.0.12,5555.0,53.0,"[192.0.0.12, 192.168.1.33]",udp,success,4.0,"[connection, denied]","%ASA-4-106023: Deny udp src dmz:192.168.1.33/5555 dst outside:192.0.0.12/53 by access-group ""dmz"" [0x123a465e, 0x4c7bf613]",warning


### Analysis 2: Port 137 - NetBIOS Traffic from External
Analyzing NetBIOS traffic from external sources, which is often associated with reconnaissance or exploitation attempts.

In [91]:
# Filter for port 137 NetBIOS traffic
netbios_traffic = df1_clean[
    (df1_clean['source_port'] == 137) | (df1_clean['destination_port'] == 137)
].copy()

print(f"Total NetBIOS (port 137) events: {len(netbios_traffic)}")
print(f"\nBreakdown by outcome:")
print(netbios_traffic.groupby('outcome').size().sort_values(ascending=False))
print(f"\nUnique source IPs:")
print(netbios_traffic['source'].unique())
print("\n" + "="*60)
print("Sample NetBIOS events:")
print("="*60)
netbios_traffic[['timestamp', 'source', 'destination', 'source_port', 'destination_port', 'related_ip', 'protocol', 'outcome', 'severity', 'event_type', 'original_event', 'log_level']]

Total NetBIOS (port 137) events: 1

Breakdown by outcome:
outcome
success    1
dtype: int64

Unique source IPs:
['192.0.2.66']

Sample NetBIOS events:


,timestamp,source,destination,source_port,destination_port,related_ip,protocol,outcome,severity,event_type,original_event,log_level
27,2013-04-30 11:23:02+00:00,192.0.2.66,10.1.2.42,137.0,137.0,"[10.1.2.42, 192.0.2.66]",udp,success,2.0,"[connection, denied]",%ASA-2-106006: Deny inbound UDP from 192.0.2.66/137 to 10.1.2.42/137 on interface inside,critical


### Analysis 3: Blacklisted/Malware Traffic
Identifying traffic flagged as blacklisted or malware-related based on the original event log.

In [92]:
# Filter for blacklist or malware related events
malware_traffic = df1_clean[
    df1_clean['original_event'].str.contains('blacklist|malware', case=False, na=False)
].copy()

print(f"Total blacklist/malware events: {len(malware_traffic)}")
print(f"\nBreakdown by outcome:")
print(malware_traffic.groupby('outcome').size().sort_values(ascending=False))
print(f"\nBreakdown by severity:")
print(malware_traffic.groupby('severity').size().sort_values(ascending=False))
print(f"\nUnique source IPs involved:")
print(malware_traffic['source'].unique())
print("\n" + "="*60)
print("Sample blacklist/malware events:")
print("="*60)

malware_traffic[['timestamp', 'source', 'destination', 'source_port', 'destination_port', 'related_ip', 'protocol', 'outcome', 'severity', 'event_type', 'original_event', 'log_level']]

Total blacklist/malware events: 2

Breakdown by outcome:
outcome
failure    1
success    1
dtype: int64

Breakdown by severity:
severity
4.0    2
dtype: int64

Unique source IPs involved:
['10.1.1.1']

Sample blacklist/malware events:


,timestamp,source,destination,source_port,destination_port,related_ip,protocol,outcome,severity,event_type,original_event,log_level
66,2015-01-14 15:16:14+00:00,10.1.1.1,192.0.2.223,33340.0,80.0,"[10.1.1.1, 10.2.1.1, 192.0.2.223]",tcp,success,4.0,[info],"%ASA-4-338004: Dynamic Filter monitored blacklisted TCP traffic from inside:10.1.1.1/33340 (10.2.1.1/33340) to outsidet:192.0.2.223/80 (192.0.2.223/80), destination 192.0.2.223 resolved from dynamic list: 192.0.2.223/255.255.255.255, threat-level: very-high, category: Malware",warning
67,2015-01-14 15:16:14+00:00,10.1.1.1,192.0.2.223,33340.0,80.0,"[10.1.1.1, 10.2.1.1, 192.0.2.223]",tcp,failure,4.0,"[connection, denied]","%ASA-4-338008: Dynamic Filter dropped blacklisted TCP traffic from inside:10.1.1.1/33340 (10.2.1.1/33340) to outsidet:192.0.2.223/80 (192.0.2.223/80), destination 192.0.2.223 resolved from dynamic list: 192.0.2.223/255.255.255.255, threat-level: very-high, category: Malware",warning


### Analysis 4: Non-Standard Port 40443 Outbound Traffic
Analyzing traffic on port 40443, which is not a commonly used port and may indicate suspicious outbound activity.

In [93]:
# Filter for port 40443 traffic (non-standard port)
port_40443_traffic = df1_clean[
    (df1_clean['source_port'] == 40443) | (df1_clean['destination_port'] == 40443)
].copy()

print(f"Total port 40443 events: {len(port_40443_traffic)}")
print(f"\nBreakdown by outcome:")
print(port_40443_traffic.groupby('outcome').size().sort_values(ascending=False))
print(f"\nBreakdown by protocol:")
print(port_40443_traffic.groupby('protocol').size().sort_values(ascending=False))
print(f"\nUnique source -> destination pairs:")
for idx, row in port_40443_traffic.iterrows():
    print(f"  {row['source']}:{row['source_port']} -> {row['destination']}:{row['destination_port']}")
print("\n" + "="*60)
print("All port 40443 events:")
print("="*60)
port_40443_traffic[['timestamp', 'source', 'destination', 'source_port', 'destination_port', 'related_ip', 'protocol', 'outcome', 'severity', 'event_type', 'original_event', 'log_level']]

Total port 40443 events: 8

Breakdown by outcome:
outcome
success    8
dtype: int64

Breakdown by protocol:
protocol
tcp    8
dtype: int64

Unique source -> destination pairs:
  10.0.0.46:49734.0 -> 192.0.0.88:40443.0
  10.0.0.46:49735.0 -> 192.0.0.88:40443.0
  10.0.0.46:49736.0 -> 192.0.0.88:40443.0
  10.0.0.46:49737.0 -> 192.0.0.88:40443.0
  10.0.0.46:49738.0 -> 192.0.0.88:40443.0
  10.0.0.46:49746.0 -> 192.0.0.88:40443.0
  10.0.0.46:49776.0 -> 192.0.0.88:40443.0
  10.0.0.46:49840.0 -> 192.0.0.88:40443.0

All port 40443 events:


,timestamp,source,destination,source_port,destination_port,related_ip,protocol,outcome,severity,event_type,original_event,log_level
18,2013-04-30 11:22:38+00:00,10.0.0.46,192.0.0.88,49734.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in permitted tcp inside/10.0.0.46(49734) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
19,2013-04-30 11:22:39+00:00,10.0.0.46,192.0.0.88,49735.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in permitted tcp inside/10.0.0.46(49735) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
20,2013-04-30 11:22:39+00:00,10.0.0.46,192.0.0.88,49736.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in permitted tcp inside/10.0.0.46(49736) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
21,2013-04-30 11:22:39+00:00,10.0.0.46,192.0.0.88,49737.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in permitted tcp inside/10.0.0.46(49737) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
22,2013-04-30 11:22:40+00:00,10.0.0.46,192.0.0.88,49738.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in permitted tcp inside/10.0.0.46(49738) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
23,2013-04-30 11:22:41+00:00,10.0.0.46,192.0.0.88,49746.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in permitted tcp inside/10.0.0.46(49746) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
30,2013-04-30 11:23:08+00:00,10.0.0.46,192.0.0.88,49776.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in permitted tcp inside/10.0.0.46(49776) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
36,2013-04-30 11:23:43+00:00,10.0.0.46,192.0.0.88,49840.0,40443.0,"[10.0.0.46, 192.0.0.88]",tcp,success,5.0,"[allowed, connection]","%ASA-5-106100: access-list acl_in est-allowed tcp inside/10.0.0.46(49840) -> outside/192.0.0.88(40443) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification


### Analysis 5: IP Spoofing Attempts (Source IP 0.0.0.0)
Identifying potential IP spoofing attempts where the source IP is 0.0.0.0, which is typically invalid for normal traffic.

In [94]:
# Filter for IP spoofing attempts (source IP = 0.0.0.0)
ip_spoofing = df1_clean[
    df1_clean['source'] == '0.0.0.0'
].copy()

print(f"Total IP spoofing attempts (source = 0.0.0.0): {len(ip_spoofing)}")
print(f"\nBreakdown by outcome:")
print(ip_spoofing.groupby('outcome').size().sort_values(ascending=False))
print(f"\nBreakdown by destination:")
print(ip_spoofing.groupby('destination').size().sort_values(ascending=False))
print(f"\nBreakdown by destination port:")
print(ip_spoofing.groupby('destination_port').size().sort_values(ascending=False))
print("\n" + "="*60)
print("All IP spoofing events:")
print("="*60)
ip_spoofing[['timestamp', 'source', 'destination', 'source_port', 'destination_port', 'related_ip', 'protocol', 'outcome', 'severity', 'event_type', 'original_event', 'log_level']]

Total IP spoofing attempts (source = 0.0.0.0): 8

Breakdown by outcome:
outcome
success    8
dtype: int64

Breakdown by destination:
destination
192.88.99.47     3
192.88.99.57     3
192.168.1.255    2
dtype: int64

Breakdown by destination port:
Series([], dtype: int64)

All IP spoofing events:


,timestamp,source,destination,source_port,destination_port,related_ip,protocol,outcome,severity,event_type,original_event,log_level
54,2014-09-12 08:50:53+00:00,0.0.0.0,192.88.99.47,NaN,NaN,"[0.0.0.0, 192.88.99.47]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.88.99.47 on interface Mobile_Traffic,critical
55,2014-09-12 08:51:01+00:00,0.0.0.0,192.88.99.57,NaN,NaN,"[0.0.0.0, 192.88.99.57]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.88.99.57 on interface Mobile_Traffic,critical
56,NaT,0.0.0.0,192.88.99.47,NaN,NaN,"[0.0.0.0, 192.88.99.47]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.88.99.47 on interface Mobile_Traffic,critical
57,2014-09-12 08:51:05+00:00,0.0.0.0,192.88.99.47,NaN,NaN,"[0.0.0.0, 192.88.99.47]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.88.99.47 on interface Mobile_Traffic,critical
58,2014-09-12 08:51:06+00:00,0.0.0.0,192.88.99.57,NaN,NaN,"[0.0.0.0, 192.88.99.57]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.88.99.57 on interface Mobile_Traffic,critical
59,2014-09-12 08:51:17+00:00,0.0.0.0,192.88.99.57,NaN,NaN,"[0.0.0.0, 192.88.99.57]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.88.99.57 on interface Mobile_Traffic,critical
60,2014-09-12 08:52:48+00:00,0.0.0.0,192.168.1.255,NaN,NaN,"[0.0.0.0, 192.168.1.255]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.168.1.255 on interface Mobile_Traffic,critical
61,2014-09-12 08:53:00+00:00,0.0.0.0,192.168.1.255,NaN,NaN,"[0.0.0.0, 192.168.1.255]",NaN,success,2.0,"[connection, denied]",%ASA-2-106016: Deny IP spoof from (0.0.0.0) to 192.168.1.255 on interface Mobile_Traffic,critical


### Analysis 6: High Severity Events (Severity > 3)
Identifying high severity events that require immediate attention or further investigation.

In [95]:
# Filter for high severity events (severity > 3)
high_severity = df1_clean[
    df1_clean['severity'] >= 5
].copy()

print(f"Total high severity events (severity > 3): {len(high_severity)}")
print(f"\nBreakdown by severity level:")
print(high_severity.groupby('severity').size().sort_values(ascending=False))
print(f"\nBreakdown by outcome:")
print(high_severity.groupby('outcome').size().sort_values(ascending=False))
# print(f"\nBreakdown by event type:")
# print(high_severity.groupby('event_type').size().sort_values(ascending=False))
# print(f"\nTop source IPs in high severity events:")
print(high_severity.groupby('source').size().sort_values(ascending=False).head(10))
print("\n" + "="*60)
print("Sample high severity events:")
print("="*60)
high_severity[['timestamp', 'source', 'destination', 'source_port', 'destination_port', 'related_ip', 'protocol', 'outcome', 'severity', 'event_type', 'original_event', 'log_level']]

Total high severity events (severity > 3): 63

Breakdown by severity level:
severity
6.0    35
5.0    28
dtype: int64

Breakdown by outcome:
outcome
success    28
dtype: int64
source
192.0.2.222      13
10.0.0.16         9
89.160.20.156     8
10.0.0.46         8
10.123.3.42       2
1.128.3.4         2
192.168.0.1       2
10.123.1.35       1
10.0.0.13         1
10.1.2.16         1
dtype: int64

Sample high severity events:


,timestamp,source,destination,source_port,destination_port,related_ip,protocol,outcome,severity,event_type,original_event,log_level
2,2014-04-15 13:34:34+00:00,10.1.2.16,192.0.0.89,2241.0,2000.0,"[10.1.2.16, 192.0.0.89]",tcp,success,5.0,"[allowed, connection]","%ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-cnt 1 first hit [0x71a87d94, 0x0]",notification
3,2013-04-24 18:00:28+00:00,172.29.2.101,192.0.2.10,1039.0,53.0,"[172.29.2.101, 192.0.2.10]",udp,success,6.0,"[connection, denied]","%ASA-6-106100: access-list inside denied udp inside/172.29.2.101(1039) -> outside/192.0.2.10(53) hit-cnt 1 first hit [0xd820e56a, 0x0]",informational
4,2013-04-24 18:00:27+00:00,172.29.2.3,192.0.2.57,1065.0,53.0,"[172.29.2.3, 192.0.2.57]",udp,success,6.0,"[allowed, connection]","%ASA-6-106100: access-list inside permitted udp inside/172.29.2.3(1065) -> outside/192.0.2.57(53) hit-cnt 144 300-second interval [0xe982c7a4, 0x0]",informational
5,2013-04-29 14:59:50+00:00,10.123.3.42,192.0.2.130,4952.0,12834.0,"[10.123.3.42, 192.0.2.130]",tcp,NaN,6.0,[info],%ASA-6-305011: Built dynamic TCP translation from outside:10.123.3.42/4952 to outside:192.0.2.130/12834,informational
6,2013-04-29 14:59:50+00:00,192.0.2.43,10.123.3.42,443.0,4952.0,"[10.123.3.42, 192.0.2.43]",tcp,NaN,6.0,[info],%ASA-6-302013: Built outbound TCP connection 89743274 for outside:192.0.2.43/443 (192.0.2.43/443) to outside:10.123.3.42/4952 (10.123.3.42/12834),informational
7,2013-04-29 14:59:50+00:00,10.123.1.35,192.0.2.130,52925.0,25882.0,"[10.123.1.35, 192.0.2.130]",udp,NaN,6.0,[info],%ASA-6-305011: Built dynamic UDP translation from outside:10.123.1.35/52925 to outside:192.0.2.130/25882,informational
8,2013-04-29 14:59:50+00:00,192.0.2.222,10.123.1.35,53.0,52925.0,"[10.123.1.35, 192.0.2.222, 192.0.2.43]",udp,NaN,6.0,[info],%ASA-6-302015: Built outbound UDP connection 89743275 for outside:192.0.2.222/53 (192.0.2.43/53) to outside:10.123.1.35/52925 (10.123.1.35/25882),informational
9,2013-04-29 14:59:50+00:00,10.123.3.42,192.0.2.130,4953.0,45392.0,"[10.123.3.42, 192.0.2.130]",tcp,NaN,6.0,[info],%ASA-6-305011: Built dynamic TCP translation from outside:10.123.3.42/4953 to outside:192.0.2.130/45392,informational
10,2013-04-29 14:59:50+00:00,192.0.2.1,10.123.3.42,80.0,4953.0,"[10.123.3.130, 10.123.3.42, 192.0.2.1]",tcp,NaN,6.0,[info],%ASA-6-302013: Built outbound TCP connection 89743276 for outside:192.0.2.1/80 (192.0.2.1/80) to outside:10.123.3.42/4953 (10.123.3.130/45392),informational
11,2013-04-29 14:59:50+00:00,192.0.2.222,10.123.1.35,53.0,52925.0,"[10.123.1.35, 192.0.2.222]",udp,NaN,6.0,"[connection, end]",%ASA-6-302016: Teardown UDP connection 89743275 for outside:192.0.2.222/53 to inside:10.123.1.35/52925 duration 1:23:45 bytes 140,informational


### Summary: All Security Issues Combined (This is not a good summay Remove Later)

In [96]:
# Summary of all security issues
summary_data = {
    'Security Issue': [
        'DNS Traffic Denials (Port 53)',
        'NetBIOS Traffic (Port 137)',
        'Blacklisted/Malware Traffic',
        'Non-Standard Port 40443',
        'IP Spoofing (Source 0.0.0.0)',
        'High Severity Events (>=5)'
    ],
    'Event Count': [
        len(dns_denied),
        len(netbios_traffic),
        len(malware_traffic),
        len(port_40443_traffic),
        len(ip_spoofing),
        len(high_severity)
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df['Percentage of Total'] = (summary_df['Event Count'] / len(df1_clean) * 100).round(2)

print("="*60)
print("SECURITY ISSUES SUMMARY")
print("="*60)
print(f"Total events in dataset 1: {len(df1_clean)}")
print("\n")
print(summary_df.to_string(index=False))
print("\n" + "="*60)

SECURITY ISSUES SUMMARY
Total events in dataset 1: 89


               Security Issue  Event Count  Percentage of Total
DNS Traffic Denials (Port 53)           13                14.61
   NetBIOS Traffic (Port 137)            1                 1.12
  Blacklisted/Malware Traffic            2                 2.25
      Non-Standard Port 40443            8                 8.99
 IP Spoofing (Source 0.0.0.0)            8                 8.99
   High Severity Events (>=5)           63                70.79



# # DataFrame 2 Analysis
- This seems like a small traffic dataset
- For the last 50 rows, we select the relevant columns and rename them for consistency.
- We also drop the original timestamp columns since we have the normalized timestamp.
- Finally, we display the cleaned dataset to verify the changes.
## Issues noted on first glance with help of AI:
- Check High Risk Malware Ports [4444,1234,5678,137]
- Where action is deny
- large data transfers > 10000 bytes

### Analysis 1: High Risk Malware Ports
Analyzing traffic on commonly exploited ports (4444, 1234, 5678, 137) which are associated with trojans, backdoors, and malware.

In [97]:
# Filter for high risk malware ports [4444, 1234, 5678, 137]
high_risk_ports = [4444, 1234, 5678, 137]

malware_ports_traffic = df2_clean[
    (df2_clean['src_port'].isin(high_risk_ports)) | 
    (df2_clean['dest_port'].isin(high_risk_ports))
].copy()

print(f"Total high risk port events: {len(malware_ports_traffic)}")
print(f"\nBreakdown by port (source):")
print(malware_ports_traffic[malware_ports_traffic['src_port'].isin(high_risk_ports)].groupby('src_port').size().sort_values(ascending=False))
print(f"\nBreakdown by port (destination):")
print(malware_ports_traffic[malware_ports_traffic['dest_port'].isin(high_risk_ports)].groupby('dest_port').size().sort_values(ascending=False))
print(f"\nBreakdown by action:")
print(malware_ports_traffic.groupby('action').size().sort_values(ascending=False))
print(f"\nBreakdown by protocol:")
print(malware_ports_traffic.groupby('protocol').size().sort_values(ascending=False))
print("\n" + "="*60)
print("All high risk port events:")
print("="*60)
malware_ports_traffic

Total high risk port events: 5

Breakdown by port (source):
Series([], dtype: int64)

Breakdown by port (destination):
dest_port
4444.0    5
dtype: int64

Breakdown by action:
action
ALLOW    5
dtype: int64

Breakdown by protocol:
protocol
TCP    5
dtype: int64

All high risk port events:


,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
129,2026-02-02 19:23:49.974906+00:00,10.0.0.30,91.214.124.143,44260.0,4444.0,TCP,ALLOW,20426.0
130,2026-02-02 19:24:49.974906+00:00,10.0.0.30,91.214.124.143,57006.0,4444.0,TCP,ALLOW,41067.0
131,2026-02-02 19:25:49.974906+00:00,10.0.0.30,91.214.124.143,54859.0,4444.0,TCP,ALLOW,26374.0
132,2026-02-02 19:26:49.974906+00:00,10.0.0.30,91.214.124.143,45670.0,4444.0,TCP,ALLOW,38047.0
133,2026-02-02 19:27:49.974906+00:00,10.0.0.30,91.214.124.143,41554.0,4444.0,TCP,ALLOW,22223.0


### Analysis 2: Denied Actions
Analyzing all traffic where the firewall action was 'deny', indicating blocked/rejected connection attempts.

In [98]:
# Filter for denied actions
denied_traffic = df2_clean[
    df2_clean['action'].str.lower() == 'deny'
].copy()

print(f"Total denied events: {len(denied_traffic)}")
print(f"\nBreakdown by protocol:")
print(denied_traffic.groupby('protocol').size().sort_values(ascending=False))
print(f"\nBreakdown by destination port:")
print(denied_traffic.groupby('dest_port').size().sort_values(ascending=False).head(10))
print(f"\nTop source IPs with denials:")
print(denied_traffic.groupby('src_ip').size().sort_values(ascending=False).head(10))
print(f"\nTop destination IPs with denials:")
print(denied_traffic.groupby('dest_ip').size().sort_values(ascending=False).head(10))
print("\n" + "="*60)
print("Sample denied events:")
print("="*60)
denied_traffic.head(20)

Total denied events: 10

Breakdown by protocol:
protocol
TCP    10
dtype: int64

Breakdown by destination port:
dest_port
22.0    10
dtype: int64

Top source IPs with denials:
src_ip
185.220.101.45    10
dtype: int64

Top destination IPs with denials:
dest_ip
10.0.0.20    10
dtype: int64

Sample denied events:


,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
119,2026-02-02 18:23:49.974906+00:00,185.220.101.45,10.0.0.20,49623.0,22.0,TCP,DENY,0.0
120,2026-02-02 18:24:49.974906+00:00,185.220.101.45,10.0.0.20,51217.0,22.0,TCP,DENY,0.0
121,2026-02-02 18:25:49.974906+00:00,185.220.101.45,10.0.0.20,41992.0,22.0,TCP,DENY,0.0
122,2026-02-02 18:26:49.974906+00:00,185.220.101.45,10.0.0.20,47886.0,22.0,TCP,DENY,0.0
123,2026-02-02 18:27:49.974906+00:00,185.220.101.45,10.0.0.20,50728.0,22.0,TCP,DENY,0.0
124,2026-02-02 18:28:49.974906+00:00,185.220.101.45,10.0.0.20,41652.0,22.0,TCP,DENY,0.0
125,2026-02-02 18:29:49.974906+00:00,185.220.101.45,10.0.0.20,48823.0,22.0,TCP,DENY,0.0
126,2026-02-02 18:30:49.974906+00:00,185.220.101.45,10.0.0.20,62080.0,22.0,TCP,DENY,0.0
127,2026-02-02 18:31:49.974906+00:00,185.220.101.45,10.0.0.20,56059.0,22.0,TCP,DENY,0.0
128,2026-02-02 18:32:49.974906+00:00,185.220.101.45,10.0.0.20,36322.0,22.0,TCP,DENY,0.0


### Analysis 3: Large Data Transfers
Analyzing connections with large data transfers (> 10,000 bytes) which may indicate data exfiltration or suspicious downloads.

In [99]:
# Filter for large data transfers (> 10000 bytes)
large_transfers = df2_clean[
    df2_clean['bytes'] > 10000
].copy()

print(f"Total large transfer events (> 10,000 bytes): {len(large_transfers)}")
print(f"\nBreakdown by action:")
print(large_transfers.groupby('action').size().sort_values(ascending=False))
print(f"\nBreakdown by protocol:")
print(large_transfers.groupby('protocol').size().sort_values(ascending=False))
print(f"\nData transfer statistics:")
print(large_transfers['bytes'].describe())
print(f"\nTop 10 largest transfers:")
print(large_transfers.nlargest(10, 'bytes')[['timestamp', 'src_ip', 'dest_ip', 'dest_port', 'protocol', 'action', 'bytes']])
print("\n" + "="*60)
print("All large transfer events:")
print("="*60)
large_transfers

Total large transfer events (> 10,000 bytes): 10

Breakdown by action:
action
ALLOW    10
dtype: int64

Breakdown by protocol:
protocol
TCP    10
dtype: int64

Data transfer statistics:
count        10.000000
mean      64169.300000
std       37038.568343
min       20426.000000
25%       29292.250000
50%       67476.000000
75%       97866.500000
max      104513.000000
Name: bytes, dtype: float64

Top 10 largest transfers:
                           timestamp     src_ip         dest_ip  dest_port  \
137 2026-02-02 20:26:49.974906+00:00   10.0.0.5   172.217.3.110      443.0   
135 2026-02-02 20:24:49.974906+00:00   10.0.0.5   172.217.3.110      443.0   
136 2026-02-02 20:25:49.974906+00:00   10.0.0.5   172.217.3.110      443.0   
138 2026-02-02 20:27:49.974906+00:00   10.0.0.5   172.217.3.110      443.0   
134 2026-02-02 20:23:49.974906+00:00   10.0.0.5   172.217.3.110      443.0   
130 2026-02-02 19:24:49.974906+00:00  10.0.0.30  91.214.124.143     4444.0   
132 2026-02-02 19:26:49.97490

,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
129,2026-02-02 19:23:49.974906+00:00,10.0.0.30,91.214.124.143,44260.0,4444.0,TCP,ALLOW,20426.0
130,2026-02-02 19:24:49.974906+00:00,10.0.0.30,91.214.124.143,57006.0,4444.0,TCP,ALLOW,41067.0
131,2026-02-02 19:25:49.974906+00:00,10.0.0.30,91.214.124.143,54859.0,4444.0,TCP,ALLOW,26374.0
132,2026-02-02 19:26:49.974906+00:00,10.0.0.30,91.214.124.143,45670.0,4444.0,TCP,ALLOW,38047.0
133,2026-02-02 19:27:49.974906+00:00,10.0.0.30,91.214.124.143,41554.0,4444.0,TCP,ALLOW,22223.0
134,2026-02-02 20:23:49.974906+00:00,10.0.0.5,172.217.3.110,51184.0,443.0,TCP,ALLOW,93885.0
135,2026-02-02 20:24:49.974906+00:00,10.0.0.5,172.217.3.110,55418.0,443.0,TCP,ALLOW,100204.0
136,2026-02-02 20:25:49.974906+00:00,10.0.0.5,172.217.3.110,43717.0,443.0,TCP,ALLOW,98256.0
137,2026-02-02 20:26:49.974906+00:00,10.0.0.5,172.217.3.110,50093.0,443.0,TCP,ALLOW,104513.0
138,2026-02-02 20:27:49.974906+00:00,10.0.0.5,172.217.3.110,58238.0,443.0,TCP,ALLOW,96698.0


### Summary: All Security Issues Combined (Dataset 2)

In [100]:
# Summary of all security issues for dataset 2
summary_data_df2 = {
    'Security Issue': [
        'High Risk Malware Ports [4444,1234,5678,137]',
        'Denied Actions',
        'Large Data Transfers (>10,000 bytes)'
    ],
    'Event Count': [
        len(malware_ports_traffic),
        len(denied_traffic),
        len(large_transfers)
    ]
}

summary_df2 = pd.DataFrame(summary_data_df2)
summary_df2['Percentage of Total'] = (summary_df2['Event Count'] / len(df2_clean) * 100).round(2)

print("="*60)
print("SECURITY ISSUES SUMMARY - Dataset 2")
print("="*60)
print(f"Total events in dataset 2: {len(df2_clean)}")
print("\n")
print(summary_df2.to_string(index=False))
print("\n" + "="*60)

SECURITY ISSUES SUMMARY - Dataset 2
Total events in dataset 2: 50


                              Security Issue  Event Count  Percentage of Total
High Risk Malware Ports [4444,1234,5678,137]            5                 10.0
                              Denied Actions           10                 20.0
        Large Data Transfers (>10,000 bytes)           10                 20.0

